In [ ]:
from scripts import mapping

In [ ]:

# Créer une carte centrée sur la France
carte = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter la couche choroplétique pour les régions
folium.Choropleth(
    geo_data=regions_geojson,
    name="Régions de France",
    fill_color="YlGnBu",  # Palette de couleurs
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Régions de France",
).add_to(carte)

# Afficher la carte
carte

In [ ]:
# Charger le fichier GeoJSON dans un GeoDataFrame
gdf_regions = gpd.GeoDataFrame.from_features(regions_geojson['features'])

# Calculer le prix moyen par région
prix_moyen_region = df_geolocalisees2.groupby('regionName')['valeur_fonciere'].sum() / df_geolocalisees2.groupby('regionName')['surface_reelle_bati'].sum()
prix_moyen_region = prix_moyen_region.reset_index()
prix_moyen_region.columns = ['nom', 'prix_moyen']

# Fusionner les données de prix avec les données géographiques
gdf_regions = gdf_regions.merge(prix_moyen_region, on='nom', how='left')

# Créer une carte centrée sur la France
carte = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter la couche choroplétique pour les régions
choropleth = folium.Choropleth(
    geo_data=regions_geojson,
    name="Régions de France",
    fill_color="YlGnBu",  # Palette de couleurs
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Prix moyen par région",
    data=prix_moyen_region,
    columns=['nom', 'prix_moyen'],
    key_on="feature.properties.nom",
).add_to(carte)

# Ajouter des popups pour afficher le nom de la région et le prix moyen
for _, row in gdf_regions.iterrows():
    folium.Marker(
        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
        popup=f"Région : {row['nom']}<br>Prix moyen : {row['prix_moyen']:.2f} €/m²",
    ).add_to(carte)

# Afficher la carte
carte

In [ ]:
# Charger le fichier GeoJSON des départements
url_departement = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
response = rq.get(url_departement)
departements_geojson = response.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_departements = gpd.GeoDataFrame.from_features(departements_geojson['features'])

# Calculer le prix moyen par département
prix_moyen_departement = df_geolocalisees2.groupby('departmentName')['valeur_fonciere'].sum() / df_geolocalisees2.groupby('departmentName')['surface_reelle_bati'].sum()
prix_moyen_departement = prix_moyen_departement.reset_index()
prix_moyen_departement.columns = ['nom', 'prix_moyen']

# En particulier, la base DVF ne contient pas de mutation en Haut-Rhin, Bas-Rhin, Moselle
prix_moyen_departement = prix_moyen_departement.dropna(subset=['prix_moyen'])

# Fusionner les données de prix avec les données géographiques des départements
gdf_departements = gdf_departements.merge(prix_moyen_departement, on='nom', how='left')

# Créer une carte centrée sur la France
carte_departements = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter la couche choroplétique pour les départements, en excluant ceux sans prix moyen
choropleth_departements = folium.Choropleth(
    geo_data=departements_geojson,
    name="Départements de France",
    fill_color="YlGnBu",  # Palette de couleurs
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Prix moyen par département",
    data=prix_moyen_departement,
    columns=['nom', 'prix_moyen'],
    key_on="feature.properties.nom",
    # Ne pas colorier les départements avec NaN dans le prix_moyen
    nan_fill_color="lightgrey"  # Couleur de remplissage pour les départements sans prix moyen
).add_to(carte_departements)

# Ajouter des popups pour afficher le nom du département et le prix moyen
for _, row in gdf_departements.iterrows():
    if pd.notna(row['prix_moyen']):  # Ne pas ajouter de popup pour les départements sans prix moyen
        folium.Marker(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            popup=f"Département : {row['nom']}<br>Prix moyen : {row['prix_moyen']:.2f} €/m²",
        ).add_to(carte_departements)

# Afficher la carte
carte_departements

In [ ]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = rq.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])

# Sélectionner les colonnes 'code' et 'nom'
code_nom = gdf_communes[['code', 'nom']]

# Renommer la colonne 'nom' en 'communeName'
code_nom = code_nom.rename(columns={'nom': 'communeName'})

# Exporter le résultat dans un fichier CSV local (UTILE SEULEMENT LA 1ère FOIS)
# code_nom.to_csv('communes_code.csv', index=False, sep=';', encoding='utf-8-sig')
# print(gdf_communes.head(10))

In [ ]:
commune = pd.read_csv('communes_code.csv', sep=';')
show(commune)
# Left_join des codes_communes avec les noms des communes geoJson
df_geolocalisees2 = df_geolocalisees2.merge(commune, left_on=["code_commune"], right_on=['code'], how='left')
df_geolocalisees2 = df_geolocalisees2.sort_values(by='code_commune', ascending=True)

In [ ]:

# Calculer le prix moyen par commune
prix_moyen_commune = df_geolocalisees3.groupby('communeName').sum()['valeur_fonciere'] / df_geolocalisees3.groupby('communeName').sum()['surface_reelle_bati']
prix_moyen_commune = prix_moyen_commune.reset_index()
prix_moyen_commune.columns = ['nom', 'prix_moyen']

nombre_transactions_commune = df_geolocalisees3.groupby('communeName').size()

nombre_transactions_commune_cotieres = nombre_transactions_commune[nombre_transactions_commune.index.isin(liste_cotieres)]

# Filtrer le DataFrame prix_moyen_commune pour ne garder que les communes présentes dans liste_commune_totale
prix_moyen_commune_filtre = prix_moyen_commune[prix_moyen_commune['nom'].isin(liste_cotieres)]

# Fusionner les données de prix avec les données géographiques des communes
gdf_communes = gdf_communes.merge(prix_moyen_commune_filtre, on='nom', how='left')

# Créer une carte centrée sur la France
carte = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter la couche choroplétique pour les communes
choropleth = folium.Choropleth(
    geo_data=communes_geojson,
    name="Communes de France",
    fill_color="YlGnBu",  # Palette de couleurs
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Prix moyen par commune",
    data=prix_moyen_commune_filtre,
    columns=['nom', 'prix_moyen'],
    key_on="feature.properties.nom",
    nan_fill_color=None,  # Spécifie qu'il ne faut pas colorier les communes sans données

).add_to(carte)

# Ajouter des popups pour afficher le nom de la commune et le prix moyen
for _, row in gdf_communes.iterrows():
    if pd.notnull(row['prix_moyen']):
        folium.Marker(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            popup=f"Commune : {row['nom']}<br>Prix moyen : {row['prix_moyen']:.2f} €/m²",
        ).add_to(carte)

# Afficher la carte
carte

In [ ]:
# Exemple d'utilisation pour la commune "Biarritz"
biarritz = mapping.create_map_for_commune(df_geolocalisees3, df_centres, 'Biarritz')
biarritz

In [ ]:

# Exemple d'utilisation pour la commune "Bandol"
heatmap = mapping.create_heatmap_for_commune(df_geolocalisees3, 'Bandol')
heatmap